In [1]:
console.log("hello");

hello


In [2]:
const tslab = require('tslab');
console.log('Hello, tslab!')
console.log('Versions:', tslab.versions)

Hello, tslab!
Versions: { tslab: '1.0.15', typescript: '4.1.2', node: 'v16.2.0' }


In [3]:
const matrix = require('node-sense-hat').Leds;

In [4]:
// To fill with a single color instead
matrix.clear([0, 1, 0]);

In [5]:
const imu = require("node-sense-hat").Imu;

const IMU = new imu.IMU();

function displayValue(err, data) {
  if (err !== null) {
    console.error("Could not read sensor data: ", err);
    return;
  }

  console.log("Accelleration is: ", JSON.stringify(data.accel, null, "  "));
  console.log("Gyroscope is: ", JSON.stringify(data.gyro, null, "  "));
  console.log("Compass is: ", JSON.stringify(data.compass, null, "  "));
  console.log("Fusion data is: ", JSON.stringify(data.fusionPose, null, "  "));
  

  console.log("Temp is: ", data.temperature);
  console.log("Pressure is: ", data.pressure);
  console.log("Humidity is: ", data.humidity);
  
};
IMU.getValue(displayValue);

Accelleration is:  {
  "x": 0.07807999849319458,
  "y": -0.07002799957990646,
  "z": 0.9864919781684875
}
Gyroscope is:  {
  "x": 0.01618792861700058,
  "y": 0.004581489134579897,
  "z": 0.018936822190880775
}
Compass is:  {
  "x": 2.2260000705718994,
  "y": 2.069200277328491,
  "z": 12.241600036621094
}
Fusion data is:  {
  "x": -0.07086800783872604,
  "y": -0.07878704369068146,
  "z": -1.1620200872421265
}
Temp is:  39.39264678955078
Pressure is:  1000.621826171875
Humidity is:  36.413307189941406


In [6]:
interface Entry {
   timestamp: Date;
   value: number;
}

var points = Array<Entry>();

In [7]:


function addToArray(err, data) {
  if (err !== null) {
    console.error("Could not read sensor data: ", err);
    return;
  }
    
    var coeff = 1000 * 60 * 1;
    var date = new Date();  //or use any other date
    var rounded = new Date(Math.round(date.getTime() / coeff) * coeff);
    let entry = { timestamp: rounded, value: data.temperature } as Entry;
    points.push(entry);
};

In [8]:
IMU.getValue(addToArray);

In [9]:
points

[ { timestamp: 2021-05-21T16:06:00.000Z, value: 39.35735321044922 } ]


In [10]:
points = []

[]


In [11]:
let timerID = setInterval(function() {
    IMU.getValue(addToArray);
}, 60 * 1000); 


In [12]:
let timeout = setTimeout(() => { clearInterval(timerID); console.log(points); }, 15 * 60 * 1000);
 // The setInterval it cleared and doesn't run anymore.

In [13]:
//clearInterval(timerID);

In [20]:
points

[
  { timestamp: 2021-05-21T16:08:00.000Z, value: 39.49852752685547 },
  { timestamp: 2021-05-21T16:09:00.000Z, value: 39.60441207885742 }
]
[
  { timestamp: 2021-05-21T16:08:00.000Z, value: 39.49852752685547 },
  { timestamp: 2021-05-21T16:09:00.000Z, value: 39.60441207885742 },
  { timestamp: 2021-05-21T16:10:00.000Z, value: 39.67500305175781 },
  { timestamp: 2021-05-21T16:11:00.000Z, value: 39.56911849975586 },
  { timestamp: 2021-05-21T16:12:00.000Z, value: 39.41029357910156 },
  { timestamp: 2021-05-21T16:13:00.000Z, value: 39.21617889404297 },
  { timestamp: 2021-05-21T16:14:00.000Z, value: 38.72206115722656 },
  { timestamp: 2021-05-21T16:15:00.000Z, value: 38.52793884277344 },
  { timestamp: 2021-05-21T16:16:00.000Z, value: 38.369117736816406 },
  { timestamp: 2021-05-21T16:17:00.000Z, value: 38.40441131591797 },
  { timestamp: 2021-05-21T16:18:00.000Z, value: 38.38676452636719 },
  { timestamp: 2021-05-21T16:19:00.000Z, value: 38.86323547363281 },
  { timestamp: 2021-05-21T16

In [21]:
const fs = require('fs');
const parse = require("csv-parse/lib/sync");
const { AnomalyDetectorClient } = require('@azure/ai-anomaly-detector');
const { AzureKeyCredential } = require('@azure/core-auth');

To create a cognitive services service and get the key
```

az group create \
    --name cognitive-services-resource-group \
    --location westus2
    
az cognitiveservices account create \
    --name anomaly-detector-resource \
    --resource-group cognitive-services-resource-group \
    --kind AnomalyDetector \
    --sku F0 \
    --location westus2 \
    --yes

az cognitiveservices account keys list \
    --name anomaly-detector-resource \
    --resource-group cognitive-services-resource-group
```

In [22]:
const dotenv = require("dotenv");
dotenv.config();
// Authentication variables
// Add your Anomaly Detector subscription key and endpoint to your environment variables.
const key = process.env["API_KEY"] || "";
const endpoint = process.env["ENDPOINT"] || "";

In [23]:
// <authentication>
let anomalyDetectorClient = new AnomalyDetectorClient(endpoint, new AzureKeyCredential(key));
// </authentication>

In [24]:
// <lastDetection>
async function lastDetection() : Promise<boolean> {

    let body = { series: points, granularity: 'minutely' }
    // Make the call to detect anomalies in the latest point of a series
    return await anomalyDetectorClient.detectLastPoint(body)
        .then((response) => {
            console.log("Latest point anomaly detection:")
            if (response.isAnomaly) {
                console.log("The latest point, in row " + points.length + ", is detected as an anomaly.")
                return true;
            } else {
                console.log("The latest point, in row " + points.length + ", is not detected as an anomaly.")
                return false;
            }
        }).catch((error) => {
            console.log(error)
            return false;
        })
}
console.log(await lastDetection());
// </lastDetection>

Latest point anomaly detection:
The latest point, in row 15, is not detected as an anomaly.
false


In [25]:
points

[
  { timestamp: 2021-05-21T16:08:00.000Z, value: 39.49852752685547 },
  { timestamp: 2021-05-21T16:09:00.000Z, value: 39.60441207885742 },
  { timestamp: 2021-05-21T16:10:00.000Z, value: 39.67500305175781 },
  { timestamp: 2021-05-21T16:11:00.000Z, value: 39.56911849975586 },
  { timestamp: 2021-05-21T16:12:00.000Z, value: 39.41029357910156 },
  { timestamp: 2021-05-21T16:13:00.000Z, value: 39.21617889404297 },
  { timestamp: 2021-05-21T16:14:00.000Z, value: 38.72206115722656 },
  { timestamp: 2021-05-21T16:15:00.000Z, value: 38.52793884277344 },
  { timestamp: 2021-05-21T16:16:00.000Z, value: 38.369117736816406 },
  { timestamp: 2021-05-21T16:17:00.000Z, value: 38.40441131591797 },
  { timestamp: 2021-05-21T16:18:00.000Z, value: 38.38676452636719 },
  { timestamp: 2021-05-21T16:19:00.000Z, value: 38.86323547363281 },
  { timestamp: 2021-05-21T16:20:00.000Z, value: 38.95146942138672 },
  { timestamp: 2021-05-21T16:21:00.000Z, value: 39.092647552490234 },
  { timestamp: 2021-05-21T16:2

In [26]:
let lastItem = points[points.length - 1]
//let entry = { timestamp: rounded, value: data.temperature } as Entry;
//points.push(entry);

In [27]:
lastItem

{ timestamp: 2021-05-21T16:22:00.000Z, value: 39.07500076293945 }


In [28]:
lastItem.timestamp.setTime(lastItem.timestamp.getTime() + 1000 * 60);

1621614180000


In [29]:
lastItem.value = lastItem.value + 100;

139.07500076293945


In [30]:
const O = [0, 0, 0];
const X = [255, 0, 0];

const cross = [
	X, O, O, O, O, O, O, X,
	O, X, O, O, O, O, X, O,
	O, O, X, O, O, X, O, O,
	O, O, O, X, X, O, O, O,
	O, O, O, X, X, O, O, O,
	O, O, X, O, O, X, O, O,
	O, X, O, O, O, O, X, O,
	X, O, O, O, O, O, O, X,
];


In [31]:
let result = await lastDetection();
if (result)
{
    matrix.setPixels(cross);
}

Latest point anomaly detection:
The latest point, in row 15, is detected as an anomaly.


In [32]:
matrix.clear([0, 0, 0]);